In [5]:
# Initialisation
import os
import requests
import json
import pandas as pd
import urllib.parse as url
from dotenv import load_dotenv

In [8]:
# Initialise panel
import panel as pn
from panel.interact import interact
from panel import widgets
pn.extension()

In [2]:
# Load env variables
load_dotenv()
api_key = os.getenv("DOMAIN_API_KEY")

In [3]:
# This is an example on how to make a request to the domain.com.au api
# First lets pretend we got an address as a full string and want to get address _suggest
address_string = "unit 15 10 12 third avenue campsie"
# The urllib.parse library lets us convert string into url text
address_url = url.quote(address_string)

# We now combine our data into the request url. The API I'm using has the below format. I set page size manually here.
# https://api.domain.com.au/v1/properties/_suggest?terms=(address)&pageSize=(number up to 20)&api_key=(your api)
# If you want to use a different get request, find the URL you need and format the request_url. If you need help you can ask me, Kevin
request_url = f"https://api.domain.com.au/v1/properties/_suggest?terms={address_url}&pageSize=1&api_key={api_key}"
get_request = requests.get(request_url)

# Data formatting
data = get_request.json()
data_indented = json.dumps(data, indent=4)
# Print data
print(data_indented)

[
    {
        "address": "15/10-12 Third Avenue, Campsie NSW 2194",
        "addressComponents": {
            "unitNumber": "15",
            "streetNumber": "10-12",
            "streetName": "Third",
            "streetType": "Av",
            "streetTypeLong": "Avenue",
            "suburb": "Campsie",
            "postCode": "2194",
            "state": "NSW"
        },
        "id": "AQ-5616-QU",
        "relativeScore": 100
    }
]


In [4]:
address_data = data[0]["addressComponents"]
# Extract location id to use as global index for all dfs regarding this location. There may be a better way to do this
location_id = data[0]["id"]
address_df = pd.DataFrame(address_data,index=[location_id])
address_df.head()

,unitNumber,streetNumber,streetName,streetType,streetTypeLong,suburb,postCode,state
AQ-5616-QU,15,10-12,Third,Av,Avenue,Campsie,2194,NSW


In [9]:
# Address locators: this request will return a unique identifier for the following address levels:
# Address, street, suburb, postcode

# https://api.domain.com.au/v1/addressLocators?searchLevel=Suburb&unitNumber=15&streetNumber=10-12&streetName=Third&streetType=Avenue&suburb=Campsie&state=NSW&postcode=2194
# Unit is not always required, but every other field is.
# If there is no unit (building is a house or w/e), unit needs to = None
level = "Address"
if "unitNumber" in address_df.columns:
    unit = address_df.iloc[0]["unitNumber"]
else:
    unit = None
street_num = address_df.iloc[0]["streetNumber"]
street_name = address_df.iloc[0]["streetName"]
street_type = address_df.iloc[0]["streetType"]
suburb = address_df.iloc[0]["suburb"]
post_code = address_df.iloc[0]["postCode"]
state = address_df.iloc[0]["state"]

# Sample of request URL. We cant use docstring as it will put \n's on every line break
"""
https://api.domain.com.au/v1/addressLocators?
searchLevel={level}&
unitNumber={unit}&
streetNumber={street_num}&
streetName={street_name}&
streetType={street_type}&
suburb={suburb}&
state={state}&
postcode={post_code}&
api_key={api_key}
"""
# Check if building is actually a unit
if not unit:
    request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"
else:
    request_url = f"https://api.domain.com.au/v1/addressLocators?searchLevel={level}&unitNumber={unit}&streetNumber={street_num}&streetName={street_name}&streetType={street_type}&suburb={suburb}&state={state}&postcode={post_code}&api_key={api_key}"
    
get_request = requests.get(request_url)

# Data formatting
id_data = get_request.json()
data_indented = json.dumps(id_data, indent=4)
# Print data
print(data_indented)

[
    {
        "types": [
            "Address"
        ],
        "addressComponents": [
            {
                "component": "StreetNumber",
                "shortName": "10-12"
            },
            {
                "component": "UnitNumber",
                "shortName": "15"
            },
            {
                "component": "StreetName",
                "shortName": "Third"
            },
            {
                "component": "StreetType",
                "shortName": "Av"
            },
            {
                "component": "Suburb",
                "shortName": "Campsie"
            },
            {
                "component": "Postcode",
                "shortName": "2194"
            },
            {
                "component": "State",
                "shortName": "NSW"
            }
        ],
        "ids": [
            {
                "level": "Address",
                "id": 5457477
            },
            {
                "level": "

In [10]:
# Get id data as df
# ID data is given as a list of dicts. To create a df out of this we need to combine the dicts into one
combined_id_data = {}
for loc_id in id_data[0]["ids"]:
    combined_id_data.update({loc_id["level"]+" id": loc_id["id"]})


id_df = pd.DataFrame(combined_id_data, index=[location_id])
id_df.head()

,Address id,Street id,Suburb id,Postcode id
AQ-5616-QU,5457477,1283352,8562,1752


In [11]:
# Theoretical example where we combined with the df above
combined_df = pd.concat([address_df, id_df], axis="columns", join="outer")
combined_df

,unitNumber,streetNumber,streetName,streetType,streetTypeLong,suburb,postCode,state,Address id,Street id,Suburb id,Postcode id
AQ-5616-QU,15,10-12,Third,Av,Avenue,Campsie,2194,NSW,5457477,1283352,8562,1752


In [12]:
# Location profiles: this request gives an info dump of a given location based on the suburb id given. 
# ID must come from Address Locator request
# https://api.domain.com.au/v1/locations/profiles/{id from any level}?api_key={api_key}
loc_id = id_df.iloc[0]["Suburb id"]

request_url = f"https://api.domain.com.au/v1/locations/profiles/{loc_id}?api_key={api_key}"
get_request = requests.get(request_url)
data = get_request.json()
data_indented = json.dumps(data, indent=4)
# Print data
print(data_indented)

# I'm going to hold off on sorting this into dfs. Theres too much data and we wont need all of it. I'll need to refer to Shavan to see what
# information we will need -Kevin

{
    "domainLocationId": 8562,
    "postcode": "2194",
    "pfLocationId": "NSW797",
    "surroundingSuburbs": [
        {
            "name": "Clemton Park",
            "urlSlug": "clemton-park-nsw-2206"
        },
        {
            "name": "Earlwood",
            "urlSlug": "earlwood-nsw-2206"
        },
        {
            "name": "Belmore",
            "urlSlug": "belmore-nsw-2192"
        },
        {
            "name": "Canterbury",
            "urlSlug": "canterbury-nsw-2193"
        },
        {
            "name": "Belfield",
            "urlSlug": "belfield-nsw-2191"
        },
        {
            "name": "Ashbury",
            "urlSlug": "ashbury-nsw-2193"
        },
        {
            "name": "Croydon Park",
            "urlSlug": "croydon-park-nsw-2133"
        }
    ],
    "urlSlug": "campsie-nsw-2194",
    "suburbName": "Campsie",
    "data": {
        "studiosForRent": 8,
        "terracedHousesForSale": 0,
        "semiDetachedHousesForSale": 0,
        "

In [13]:
# Properties: Using a given property id provided by _suggest, this request gives an info dump on the characteristics of the property
# https://api.domain.com.au/v1/properties/{property unique index}?api_key={api_key}
prop_id = address_df.index[0]

request_url = f"https://api.domain.com.au/v1/properties/{prop_id}?api_key={api_key}"
get_request = requests.get(request_url)
data = get_request.json()
data_indented = json.dumps(data, indent=4)
# Print data
print(data_indented)

# Also for this im not going to put this data into df. Need to know what info I need. Will refer to Shavan -Kevin

{
    "onMarketTypes": [],
    "status": "OffMarket",
    "address": "15/10-12 Third Avenue, Campsie NSW 2194",
    "addressCoordinate": {
        "lat": -33.90835,
        "lon": 151.09743
    },
    "addressId": 5457477,
    "areaSize": 1337,
    "bedrooms": 2,
    "created": "2021-11-07T23:04:11.523Z",
    "features": [
        "Garage"
    ],
    "flatNumber": "15",
    "gnafIds": [
        {
            "monthNo": 8,
            "yearNo": 2021,
            "gnafPID": "GANSW711374567"
        }
    ],
    "id": "AQ-5616-QU",
    "isResidential": true,
    "photos": [],
    "planNumber": "170156",
    "postcode": "2194",
    "propertyCategory": "Unit",
    "propertyCategoryId": 2,
    "state": "NSW",
    "streetAddress": "10-12 Third Avenue",
    "streetName": "Third",
    "streetNumber": "10-12",
    "streetType": "Av",
    "streetTypeLong": "Avenue",
    "suburb": "Campsie",
    "suburbId": 8562,
    "urlSlug": "15-10-12-third-avenue-campsie-nsw-2194",
    "urlSlugShort": "15-10-1

In [9]:
def text_printer(text):
    print(text)
    

interact(text_printer, text="text")

text


gwer 
wegweg 
qwdqwwdfwdgwe

Column
    [0] Column
        [0] TextInput(name='text', value='text')
    [1] Row
        [0] Str(None, name='interactive00109')

In [21]:
dashboard = pn.Column(input())
dashboard.servable()

 wegwe


Column
    [0] Markdown(str)

In [22]:
"""pipeline = pn.pipeline.Pipeline()
pipeline.add_stage("text_input", input_panel)
pipeline.add_stage("output", dashboard)"""

'pipeline = pn.pipeline.Pipeline()\npipeline.add_stage("text_input", input_panel)\npipeline.add_stage("output", dashboard)'

In [20]:
value = input()

 fwqeg
